In [1]:
import numpy as np
from PIL import Image

In [2]:
# This function take the image downloaded and resized manually of Super Mario's map
# and create a compatible RGB JPG file to use the crop function below
def createCompatibleImage(img_path):
    png = Image.open(img_path).convert('RGBA')
    background = Image.new('RGBA', png.size, (255,255,255))

    alpha_composite = Image.alpha_composite(background, png)
    alpha_composite.save('levels/mario-8-1.jpg', 'JPEG', quality=100)

In [314]:
createCompatibleImage('levels/mario-8-1.png')

In [445]:
# Now we have the image in the right format
img = np.array(Image.open('levels/mario-1-1.jpg'))

imgHeight = img.shape[0]
imgWidth = img.shape[1]

# Every tile is 16x16 pixels 
windowsize_r = 16
windowsize_c = 16

tiles = []

# Crop out the window
for r in range(0, imgHeight, windowsize_r):
    for c in range(0, imgWidth, windowsize_c):
        window = img[r: r + windowsize_r, c: c + windowsize_c]
        tile = Image.fromarray(window, 'RGB')
        tiles.append(tile)
        #img.save('my.png')
        #tile.show()


print(len(tiles))
tiles[2740]

'''I used this part of code to save locally the different tile s blocks'''
#coinBlock = tiles[862]
#blankSpace = tiles[0]
#breakableBrick = tiles[2512]
#solidBrick = tiles[2456]
#bushBlock = tiles[2456]
#floorBlock = tiles[2740]

#coinBlock.save('blocks/coinBlock.png')
#blankSpace.save('blocks/blankSpace.png')
#breakableBrick.save('blocks/breakableBrick.png')
#solidBrick.save('blocks/solidBrick.png')
#bushBlock.save('blocks/bushBlock.png')
#floorBlock.save('blocks/floorBlock.png')

2743


In [316]:
# This function takes the cropped tiles and translate them into the relative symbols (of our choice)
def image2text(tiles):
    
    counter = 0
    f = open("txtlevels/mario-8-1.txt", "w+")
    
    for tile in tiles:
        
        # If end of level, new line
        if (counter == (imgWidth)/windowsize_c):
            counter = 0
            f.write("\r\n")
          
        # Check what type of tile
        if (np.sum(np.array(tile)) == np.sum(np.array(blankSpace))):
            f.write("-")
        elif(np.sum(np.array(tile)) == np.sum(np.array(coinBlock))):
            f.write("?")
        elif(np.sum(np.array(tile)) == np.sum(np.array(breakableBrick))):
            f.write("#")
        elif(np.sum(np.array(tile)) == np.sum(np.array(solidBrick))):
            f.write("=")
        elif(np.sum(np.array(tile)) == np.sum(np.array(floorBlock))):
            f.write("f")
        else:
            f.write("-")
        
        counter += 1
    
    f.close()

In [317]:
image2text(tiles)

In [481]:
# Rotate file by 90 degrees, so that the RNN can work better because of a more significative pattern

f = open("txtlevels/mario-8-1.txt", 'r')

rows = np.array([line for line in f if line.split() != []])

f.close()

f = open("mario-8-1-columns.txt", "w+")

index = 0
while(index < (imgWidth)//windowsize_c):
    for i in range(0, len(rows)):
        f.write(rows[i][index])
    f.write("\r\n")
    index += 1
f.close() 

In [42]:
# Put togheter all the txt level files
import fileinput
import glob

file_list = glob.glob("*.txt")

with open('data.txt', 'w') as file:
    input_lines = fileinput.input(file_list)
    file.writelines(input_lines)

In [14]:
import tensorflow as tf
from tensorlm import CharLM
import random

tf.reset_default_graph()

with tf.Session() as session:
    
    # Create a new model. You can also use WordLM
    model = CharLM(session, "data.txt", max_vocab_size=13,
                   neurons_per_layer=256, num_layers=3, num_timesteps=15)
    
    # Train it 
    model.train(session, max_epochs=10, max_steps=1000)
    
    # Let it generate a text
    for i in range(10):
        seed = random.choice(open('data.txt').readlines())
        generated = model.sample(session, seed, num_steps=100)
        print(generated)

-------f

-----------f

------------f

------------f

------------f

------------f

------------f

-
-------f

-----------f

------------f

------------f

------------f

------------f

------------f

-
---------------------f

------------f

------------f

------------f

------------f

------------f

-
------f

-----------f

------------f

------------f

------------f

------------f

------------f

--
---------------------f

------------f

------------f

------------f

------------f

------------f

-
---------------------f

------------f

------------f

------------f

------------f

------------f

-
-------f

-----------f

------------f

------------f

------------f

------------f

------------f

-
-------f

-----------f

------------f

------------f

------------f

------------f

------------f

-
------f

-----------f

------------f

------------f

------------f

------------f

------------f

--
-------f

-----------f

------------f

------------f

------------f

------------f

-------